# Import all necessary packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cap-4611-2021-fall-assignment-4/sample_submission.csv
/kaggle/input/cap-4611-2021-fall-assignment-4/eval.csv
/kaggle/input/cap-4611-2021-fall-assignment-4/train.csv


# Reading Data

In [2]:
df = pd.read_csv('/kaggle/input/cap-4611-2021-fall-assignment-4/train.csv', low_memory=False) # Training Data
test_df = pd.read_csv('/kaggle/input/cap-4611-2021-fall-assignment-4/eval.csv', low_memory=False) # Testing Data
sample_df = pd.read_csv('/kaggle/input/cap-4611-2021-fall-assignment-4/sample_submission.csv', low_memory=False) # Sample Submission

# Exploratory Data Analysis

In [3]:
df.head()

,id,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,10000,4,0,0,0,0,0,0,1,1,...,111,81,0,0,0,3,28,0,0,0
1,10001,5,0,0,0,0,0,0,0,0,...,0,0,0,4,0,45,99,0,0,0
2,10002,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,6,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
4,10004,8,0,0,0,0,0,0,0,0,...,133,125,109,90,42,0,0,0,0,0


In [4]:
df.tail()

,id,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
59995,69995,4,0,0,0,0,1,2,0,0,...,0,2,0,18,142,104,61,0,0,0
59996,69996,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,69997,3,0,0,0,0,0,0,0,0,...,54,0,0,0,0,0,0,0,0,0
59998,69998,9,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
59999,69999,4,0,0,0,0,0,0,0,2,...,94,74,0,0,0,0,0,0,0,0


In [5]:
test_df.head()

,id,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,2,...,0,0,0,6,217,220,141,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,75,74,109,55,0,3,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.tail()

,id,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
9995,9995,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
9996,9996,0,0,0,0,0,0,0,0,0,...,0,0,0,9,0,164,173,0,9,0
9997,9997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,102,176,94,0,0,0
9998,9998,0,0,0,0,0,0,1,2,0,...,3,2,0,31,45,34,1,0,0,0
9999,9999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
sample_df.head()

,id,label
0,0,3
1,1,9
2,2,1
3,3,9
4,4,2


In [8]:
sample_df.tail()

,id,label
9995,9995,8
9996,9996,1
9997,9997,6
9998,9998,5
9999,9999,7


# Feature Engineering

In [9]:
hi = test_df['id']
df.drop(['id'], axis=1)
test_df.drop(['id'], axis=1)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,6,217,220,141,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,75,74,109,55,0,3,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,9,0,164,173,0,9,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,102,176,94,0,0,0
9998,0,0,0,0,0,0,1,2,0,0,...,3,2,0,31,45,34,1,0,0,0


In [10]:
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Splitting data into training and testing data

In [11]:
X = df.drop(['label'], axis=1)
Y = df['label']
x = X.values
y = Y.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)
x_train = x_train / 255
x_test = x_test / 255

# Building Sequential Model

In [12]:
model = Sequential()
model.add(Flatten(input_shape=(785,)))
model.add(Dense(128, activation='selu'))
model.add(Dense(10))
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50)
model.summary()

2021-11-23 00:34:50.467945: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 00:34:50.580343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 00:34:50.581174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 00:34:50.583135: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50
1688/1688 [==============================] - 5s 2ms/step - loss: 0.6187 - accuracy: 0.7884
Epoch 2/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4556 - accuracy: 0.8367
Epoch 3/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4113 - accuracy: 0.8524
Epoch 4/50
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3786 - accuracy: 0.8620
Epoch 5/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3569 - accuracy: 0.8697
Epoch 6/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3442 - accuracy: 0.8745
Epoch 7/50
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3272 - accuracy: 0.8787
Epoch 8/50
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3154 - accuracy: 0.8814
Epoch 9/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3033 - accuracy: 0.8876
Epoch 10/50
1688/1688 [==============================] - 3s 2ms/step - loss: 0.295

# Saving Solution to CSV

In [13]:
solution = np.array(model(test_df.to_numpy())).flatten()
output = pd.DataFrame({'id': hi, 'label': solution[:10000]})
display(output)
output.to_csv('submission.csv', index=False)

print('Your submission was successfully saved') # Asserting submission was successfully saved

,id,label
0,0,-1316.781006
1,1,-4236.448242
2,2,-358.860138
3,3,-1305.854370
4,4,645.816406
...,...,...
9995,9995,-1711.438354
9996,9996,-126.247368
9997,9997,965.221008
9998,9998,-164.432007


Your submission was successfully saved
